In [70]:
import numpy as np
import math

# https://theailearner.com/2018/12/29/image-processing-bilinear-interpolation/

In [71]:
def set_border(arr):
  row, col = arr.shape
  br = row + 2
  bc = col + 2
  border_arr = np.zeros((br, bc))
  
  for i in range(br):
    for j in range(bc):
      if i == 0 and j == 0:
        border_arr[i][j] = arr[0][0]
      elif i == 0 and j == bc - 1:
        border_arr[i][j] = arr[0][col - 1]
      elif i == br - 1 and j == 0 :
        border_arr[i][j] = arr[row - 1][0]
      elif i == br - 1 and j == bc - 1:
        border_arr[i][j] = arr[row - 1][col - 1]
      elif i == 0:
        border_arr[i][j] = arr[0][j - 1]
      elif i == br - 1:
        border_arr[i][j] = arr[row - 1][j - 1]
      elif j == 0:
        border_arr[i][j] = arr[i - 1][0]
      elif j == bc - 1:
        border_arr[i][j] = arr[i - 1][col-1]
      else:
        border_arr[i][j] = arr[i - 1][j - 1]

  return border_arr


In [72]:
def calculate_right_weight(row, col):
  dx_=np.zeros(col)
  dy_=np.zeros(row)
  scale_x = 0.5
  scale_y = 0.5
  for i in range(col):
    fx = ((i + 0.5) * scale_x - 0.5)
    sx = math.floor(fx)
    fx = fx - sx
    dx_[i] = fx
    
  for i in range(row):
    fy = ((i + 0.5) * scale_y - 0.5)
    sy = math.floor(fy)
    fy = fy - sy
    dy_[i] = fy
  return dx_, dy_

In [73]:
def calculate_pixel_val(i,j, temp, dx_, dy_):
  right_w = dx_[j]
  left_w = 1 - right_w

  x_top = (temp[0][0] * left_w) + (temp[0][1] * right_w)
  x_bottom = (temp[1][0] * left_w) + (temp[1][1] * right_w)

  right_w = dy_[i]
  left_w = 1 - right_w
  val = (x_top * left_w) + (x_bottom * right_w)
  return math.floor(val)

In [74]:
def get_upscaled_arr(arr):
  row, col = arr.shape
  new_row, new_col = row * 2, col * 2

  brr = set_border(arr)
  dx_, dy_ = calculate_right_weight(new_row, new_col)
  # print(brr)
  # print(dx_)
  # print(dy_)
  up_scaled_arr = np.zeros((new_row, new_col))

  for i in range(row + 1):
    for j in range(col + 1):
      temp = brr[i:i+2, j: j + 2]
      # print(temp)
      if i == 0 and j == 0:
        up_scaled_arr[(2 * i)][(2 * j)] = calculate_pixel_val((2 * i), (2 * j), temp, dx_, dy_)
      elif i == 0 and j == col:
        up_scaled_arr[(2 * i)][(2 * j) - 1] = calculate_pixel_val((2 * i), (2 * j) - 1, temp, dx_, dy_)
      elif i == row and j == 0:
        up_scaled_arr[(2 * i) - 1][(2 * j)] = calculate_pixel_val((2 * i) - 1, (2 * j), temp, dx_, dy_)
      elif i == row and j == col:
        up_scaled_arr[(2 * i) - 1][(2 * j)-1] = calculate_pixel_val((2 * i)-1, (2 * j) - 1, temp, dx_, dy_)
      elif i == 0:
        up_scaled_arr[(2 * i)][(2 * j) - 1] = calculate_pixel_val((2 * i), (2 * j) - 1, temp, dx_, dy_)
        up_scaled_arr[(2 * i)][(2 * j)] = calculate_pixel_val((2 * i), (2 * j), temp, dx_, dy_)
      elif i == row:
        up_scaled_arr[(2 * i) - 1][(2 * j) - 1] = calculate_pixel_val((2 * i) - 1, (2 * j) - 1, temp, dx_, dy_)
        up_scaled_arr[(2 * i) - 1][(2 * j)] = calculate_pixel_val((2 * i) - 1, (2 * j), temp, dx_, dy_)
      elif j == 0:
        up_scaled_arr[(2 * i) - 1][(2 * j)] = calculate_pixel_val((2 * i) - 1, (2 * j), temp, dx_, dy_)
        up_scaled_arr[(2 * i)][(2 * j)] = calculate_pixel_val((2 * i), (2 * j), temp, dx_, dy_)
      elif j == col:
        up_scaled_arr[(2 * i) - 1][(2 * j) - 1] = calculate_pixel_val((2 * i) - 1, (2 * j) - 1, temp, dx_, dy_)
        up_scaled_arr[(2 * i)][(2 * j) - 1] = calculate_pixel_val((2 * i), (2 * j) - 1, temp, dx_, dy_)
      else:
        up_scaled_arr[(2 * i) - 1][(2 * j) - 1] = calculate_pixel_val((2 * i) - 1, (2 * j) - 1, temp, dx_, dy_)
        up_scaled_arr[(2 * i) - 1][(2 * j)] = calculate_pixel_val((2 * i) - 1, (2 * j), temp, dx_, dy_)
        up_scaled_arr[(2 * i)][(2 * j) - 1] = calculate_pixel_val((2 * i), (2 * j) - 1, temp, dx_, dy_)
        up_scaled_arr[(2 * i)][(2 * j)] = calculate_pixel_val((2 * i), (2 * j), temp, dx_, dy_)

  # print(up_scaled_arr)
  return up_scaled_arr

In [75]:
arr = np.array([[10, 20],
                [30, 40]])
print(arr)
up_arr = get_upscaled_arr(arr)
print(up_arr)

[[10 20]
 [30 40]]
[[10. 12. 17. 20.]
 [15. 17. 22. 25.]
 [25. 27. 32. 35.]
 [30. 32. 37. 40.]]


In [76]:
crr = np.array([[10, 20, 30, 40],
                [50, 60, 70, 80],
                [90, 100, 110, 120],
                [130, 140, 150, 160]])
print(crr)
up_crr = get_upscaled_arr(crr)
print(up_crr)

[[ 10  20  30  40]
 [ 50  60  70  80]
 [ 90 100 110 120]
 [130 140 150 160]]
[[ 10.  12.  17.  22.  27.  32.  37.  40.]
 [ 20.  22.  27.  32.  37.  42.  47.  50.]
 [ 40.  42.  47.  52.  57.  62.  67.  70.]
 [ 60.  62.  67.  72.  77.  82.  87.  90.]
 [ 80.  82.  87.  92.  97. 102. 107. 110.]
 [100. 102. 107. 112. 117. 122. 127. 130.]
 [120. 122. 127. 132. 137. 142. 147. 150.]
 [130. 132. 137. 142. 147. 152. 157. 160.]]
